In [6]:
import requests as req
import pandas as pd
import json
import time

# Utils

## Consts

In [7]:
GITHUB_REST_URL = 'https://api.github.com/'
TOKEN = ''
HEADERS = {
  'Authorization': f'bearer {TOKEN}',
}

# Functions

In [8]:
def measure_rest(path: str):
    rest_endpoint = GITHUB_REST_URL + path
    start_time = time.time()
    response = req.get(rest_endpoint, headers=HEADERS)
    end_time = time.time()
    if response.status_code == 200:
        return end_time - start_time, len(response.content)
    
    raise Exception(f'Erro ao fazer requisição: {response.status_code} \n {response.text}')

## Paths

In [9]:
path_1 = 'users/Netflix/repos?per_page=5'
path_2 = 'repos/Netflix/Hystrix/issues?state=open&per_page=5'
path_3 = 'repos/Netflix/Hystrix/pulls?per_page=5'

# experiment

In [10]:
data = []
for _ in range(10):
    graphql_time, graphql_size = measure_rest(path_1)
    data.append({
        'type_query': '1',
        'graphql_time': graphql_time,
        'graphql_size': graphql_size,
    })

for _ in range(10):
    graphql_time, graphql_size = measure_rest(path_2)
    data.append({
        'type_query': '2',
        'graphql_time': graphql_time,
        'graphql_size': graphql_size,
    })

for _ in range(10):
    graphql_time, graphql_size = measure_rest(path_3)
    data.append({
        'type_query': '3',
        'graphql_time': graphql_time,
        'graphql_size': graphql_size,
    })
    
df = pd.DataFrame(data)
df.to_csv('api_rest_results.csv', index=False)